In [13]:
import os
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import sklearn.metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier


In [14]:
# Read the Semeval data and return a dataframe
def read_semeval_data(filename):
    '''
    Description: reads Semantic Evaluation XML dataset and converts into a 
                 dataframe
    Arguments:
                 filename: string with file path (including filename)
    Returns :    pandas data frame
    Important:   this function only puts positive and neutral reviews in dataframe                          
    '''
    f = open('data/train.xml', 'r')
    raw_training_data = f.read()
    xmldoc = BeautifulSoup(raw_training_data,'lxml-xml')
    sentences = xmldoc.Reviews.find_all('sentences')
    opinions = xmldoc.Reviews.find_all('Opinions')
    reviews = []
    for i in range(0,len(sentences)):
        record = {}
        entity_aspect_pairs = opinions[i].find_all('Opinion')
        for ea_pair in entity_aspect_pairs:
            ea = ea_pair.attrs['category']
            polarity = ea_pair.attrs['polarity']
            if(polarity == 'positive'):
                record[ea] = 1
            elif(polarity == 'negative'):
                record[ea] = -1
            else:
                record[ea] = 0           
        record['TEXT'] = sentences[i].get_text()
        reviews.append(record)
    #Create a dataframe
    df=pd.DataFrame(reviews)
    #Change order of the columns so that text appears first
    cols = df.columns.tolist()
    cols.sort()
    cols.reverse()
    df = df[cols]
    df.fillna(0, inplace=True)
    return df

entity_labels = ['Food','Drinks','Service','Ambience','Location','Restaurant']
<br/>attributes_labels = ['General','Prices','Quality','Style&Options','Miscellaneous']

##### Possible Combinations of Entities and Attributes #####

<img src='images/entity_attributes_combinations.jpg' style='width:50;height:50'>

In [15]:
df_training = read_semeval_data('data/train.xml')
df_testing = read_semeval_data('data/test.xml')


In the dataframe above a +1 indicates a positive polarity and -1 indicates negative polarity whereas 0 means that this aspect is not found or found to be neutral

In [16]:
#Function to clean the text data
#Remove punctuations, newline characters and convert to lowercase.
#Note that we are not removing dot to mark sentence boundary
def clean_text_data(data):
    '''
    Description: Given text returns cleaned version
    Arguments:
                  data: string with raw review text
    Returns  :
                  cleaned: string with unwanted characters removed
    '''
    prog = re.compile('[\t\n\r\f\v\d\']', re.UNICODE)
    data = re.sub(prog, ' ', data).lower()
    prog = re.compile('[!\"#$%&\'()*+\,-/:;<=>?@[\]^_`{|}~]', re.UNICODE)
    cleaned = re.sub(prog, ' ', data)
    return cleaned

In [17]:
#Function to decode and print output labels and polarity for a review
def output_to_labels(output):
    '''
     Description: Converts predicted output for a review into labels and polarity
     Arguments: output a numpy array
                
    '''
    labels = ['SERVICE#GENERAL', 'RESTAURANT#PRICES',
       'RESTAURANT#MISCELLANEOUS', 'RESTAURANT#GENERAL', 'LOCATION#GENERAL',
       'FOOD#STYLE_OPTIONS', 'FOOD#QUALITY', 'FOOD#PRICES',
       'DRINKS#STYLE_OPTIONS', 'DRINKS#QUALITY', 'DRINKS#PRICES',
       'AMBIENCE#GENERAL']
    for index in range(len(labels)):
        value = output[0,index]
        if(value == 1):
            print(labels[index],':',' ','positive')
        elif(value == -1):
            print(labels[index],':',' ','negative')
    

In [18]:
df_training['TEXT'] = df_training['TEXT'].apply(clean_text_data)

In [19]:
df_training.columns

Index(['TEXT', 'SERVICE#GENERAL', 'RESTAURANT#PRICES',
       'RESTAURANT#MISCELLANEOUS', 'RESTAURANT#GENERAL', 'LOCATION#GENERAL',
       'FOOD#STYLE_OPTIONS', 'FOOD#QUALITY', 'FOOD#PRICES',
       'DRINKS#STYLE_OPTIONS', 'DRINKS#QUALITY', 'DRINKS#PRICES',
       'AMBIENCE#GENERAL'],
      dtype='object')

In [20]:
X_train = df_training['TEXT']
y_train = df_training.drop('TEXT',axis=1)
X_test = df_testing['TEXT']
y_test = df_testing.drop('TEXT',axis=1)

In [21]:
y_train.head()

,SERVICE#GENERAL,RESTAURANT#PRICES,RESTAURANT#MISCELLANEOUS,RESTAURANT#GENERAL,LOCATION#GENERAL,FOOD#STYLE_OPTIONS,FOOD#QUALITY,FOOD#PRICES,DRINKS#STYLE_OPTIONS,DRINKS#QUALITY,DRINKS#PRICES,AMBIENCE#GENERAL
0,-1.0,0.0,0.0,-1,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,1.0,1.0,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [22]:
classifier = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=50,min_samples_leaf=2, min_samples_split=4, 
                                      max_depth=3,class_weight='balanced',oob_score=True)))])
classifier.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...
            oob_score=True, random_state=None, verbose=0, warm_start=False),
           n_jobs=1))])

In [23]:
y_predicted = classifier.predict(X_test)

In [24]:
f1_sum = 0
for column_index in range(y_test.shape[1]):
    f1=sklearn.metrics.f1_score(np.array(y_test)[:,column_index], y_predicted[:,column_index], average='macro')
    f1_sum = f1_sum + f1 

In [25]:
print(f1_sum/12)

0.86473457117


In [34]:
review_text = "This is a car"

In [35]:
dictionary = {'review':clean_text_data(review_text)}
df_input=pd.DataFrame(dictionary,index=np.arange(len(dictionary.keys())))

In [36]:
output = classifier.predict(df_input)

In [37]:
df_input.head()

,review
0,this is a car


In [38]:
output=classifier.predict(df_input['review'])

In [39]:
output_to_labels(output)

SERVICE#GENERAL :   positive
RESTAURANT#PRICES :   positive
RESTAURANT#MISCELLANEOUS :   positive
RESTAURANT#GENERAL :   positive
FOOD#QUALITY :   positive
AMBIENCE#GENERAL :   positive
